In [1]:
import torch
import torchvision

mobilenetv2 = torch.hub.load("pytorch/vision:v0.10.0", "mobilenet_v2")

Using cache found in /home/shusrith/.cache/torch/hub/pytorch_vision_v0.10.0


In [2]:
import torchvision.transforms as transforms

transform_train = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomAffine(10, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

transform_test = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [3]:
train = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train
)
test = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=False, transform=transform_test
)

Files already downloaded and verified


In [4]:
class MobNetCustom(torch.nn.Module):
    def __init__(self, model):
        super(MobNetCustom, self).__init__()
        self.features = model.features
        self.avgpool1 = torch.nn.AvgPool2d(4)
        self.fc1 = torch.nn.Linear(1280, 128)
        self.fc2 = torch.nn.Linear(128, 10)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = torch.nn.functional.relu(x)
        return self.fc2(x)

In [5]:
model = MobNetCustom(mobilenetv2)
for i in model.features.parameters():
    i.requires_grad = False

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)

In [7]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [44]:
for epoch in range(5):
    model.train()
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct, total = 0, 0
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}")

100%|██████████| 196/196 [01:05<00:00,  2.98it/s]


Accuracy: 11.65


100%|██████████| 196/196 [01:06<00:00,  2.94it/s]


Accuracy: 15.83


100%|██████████| 196/196 [01:13<00:00,  2.66it/s]


Accuracy: 14.85


100%|██████████| 196/196 [01:15<00:00,  2.59it/s]


Accuracy: 16.4


100%|██████████| 196/196 [01:16<00:00,  2.55it/s]


Accuracy: 16.91


In [8]:
for i in model.features[:10].parameters():
    i.requires_grad = True

In [ ]:
for epoch in range(5):
    model.train()
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct, total = 0, 0
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}")